# Filtro de spam


#### [Entrenamiento:](#Entrenamiento)
- def que separe una línea en palabras y las agregue a un conjunto dado por parámetro
- def de un generator que devuelva un set de palabras por cada correo de un mbox dado por parámetro
- def que devuelve Dict de la forma {'w' : n } con w = palabra y n = Hw para 'ham.mbox' o n = Sw para 'spam.box'

#### [Clasificación:](#Clasificación)
- def que devuelva P(y|xw) para cada palabra w de un conjunto de palabras, uno por cada correo de 'news.box'
- def que devuelva lista de 15 palabras de mejor clasificación individual de un correo nuevo
- def que devuelva P(y|x1, . . . , x15) de la lista anterior
- def que devuelva lista de 0 o 1 por cada correo de 'news.mbox' en el mismo orden

#### [Incorporación:](#Incorporación)
- def que devuelva un set de palabras de un 'new-spam.mbox' o 'new-ham.mbox' que contiene un solo mensaje
- def que combine el anterior conjunto al dict de entrenamiento aumentando en 1 cada valor
- def que escriba el nuevo correo 'new-spam.mbox' o 'new-ham.mbox' en el fichero que corresponda

#### [Uso:](#Uso)
- Ejemplo de uso de los 3 procedimientos anteriores

# Entrenamiento

In [2]:
import re


# Separa al string 'line' según el regex delimiter
# Y añade los trozos al set 'words' pasado por parámetro
def process_line(line, words, delimiter = '[,. ()]|\n|\t'):
    for word in re.split(delimiter, line):
        words.add(word)


# Devuelve un set de palabras diferentes de un solo correo
def words_per_mail(mbox):    
    with open(mbox) as fp:
        lines = iter(fp.readlines())
        while True:
            words = set()
            for subject in lines:
                if subject.startswith("Subject: "):
                    break
            else:
                break

            process_line(subject[9:], words)

            for status in lines:
                if status == "X-KMail-MDN-Sent:  \n":
                    break

            intro = fromOne = False
            for line in lines:
                if line == '\n' :
                    intro = True
                    continue
                elif line.startswith('From: ') and fromOne :
                    break
                elif line.startswith('From ') and intro :
                    fromOne = True
                else:
                    intro = False
                    process_line(line, words)

            words.remove('')
            yield words


# Agrega words a dictionary, incrementando la ocurrencia en 1
def merge_to_dict(words, dictionary):
    for w in words:
        dictionary[w] = dictionary[w] + 1 if w in dictionary else 1


# Devuelve dict de palabras no repetidas y el número de correos
# en los que aparecen, del mbox dado por parámetro
def training_dict(mbox):
    dictionary = {}
    n = 0
    for words in words_per_mail(mbox):
        merge_to_dict(words, dictionary)
        n += 1
    return dictionary, n



In [3]:
# Devuelve dict de la forma {'word' : Sw }
# y el número de correos para spambox
# y otro dict de la forma {'word' : Hw }
# y el número de correos para hambox
def training(spambox, hambox):
    spam_dict, S = training_dict(spambox)
    ham_dict, H = training_dict(hambox)
    return spam_dict, S, ham_dict, H


# Clasificación

In [4]:
# Devuelve generator que recorre las palabras de cada correo de newsbox
# Devuelve un set por cada llamada, que corresponde al siguiente correo
def news_words_generator(newsbox = './news.mbox'):
    return words_per_mail(newsbox)

#Devuelve P(y|xw) por cada palabra en un correo
def probPerWord(words):
    #S = número de correos spam
    #H = número de correos ham
    #spam_dict = dict de la forma {'word' : Sw }
    #ham_dict = dict de la forma {'word' : Hw }
    spam_dict, S, ham_dict, H = training('./spam.mbox', './ham.mbox')
    Py = S/(S + H)
    
    Pxwy = []
    Pw = []
    Pyxw = []
    for word in words:
        if word in spam_dict.keys():
            Sw = spam_dict[word] #número de correos Spam donde aparece esa palabra
        else:
            Sw = 0
        Pxwy.append(Sw/S)
        if word in ham_dict.keys():
            Hw = ham_dict[word] # número de correos no Spam donde aparece esa palabra
        else:
            Hw = 0
        if Sw==0 or Hw==0:
            Pw.append(0.5)
        else:
            Pw.append((Sw+Hw)/(S+H))
    #Calculamos P(y|xw) con la regla de Bayes: (P(x|y)*P(y)/(P(x|y)*P(y) + P(x|¬y)*P(¬y)))
    ##########################CALCULARLO BIEN##################################
    counter2 = 0
    for value in Pxwy:
        Pyxw.append(value*Py/Pw[counter2])
        counter2+=1
    return Pyxw
                
                

In [5]:
from collections import namedtuple

Pair = namedtuple("Pair", ["value", "position"])

#Devuelve una lista de 15 palabras de mejor clasificación individual de un correo
def bestQualificationWords(words):
    probabilidades = probPerWord(words)
    wordsList = list(words)
    diffZeroOne = []
    bestFifteen = []
    result = []
    #Calculamos la diferencia entre las probabilidades y 0 ó 1 (dependiendo de su valor), para ver cuáles son
    #los más próximos a éstos y escoger los 15 mejores
    counter = 0
    for prob in probabilidades: 
        if prob <= 0.5:
            diffZeroOne.append(Pair(abs(prob), counter))
        elif prob > 0.5:
            diffZeroOne.append(Pair(abs(1-prob), counter))
        counter+=1
    #Ordenamos de menor a mayor
    diffSorted = sorted(diffZeroOne)
    #Seleccionamos los 15 primeros (los de menor diferencia con 0 ó 1)
    i = 0
    while i<15:
        if (i < len(diffSorted)):
            bestFifteen.append(Pair(diffSorted[i].value, diffSorted[i].position))
            i+=1
    #Por último, devolvemos las palabras que se corresponden a esos resultados
    for best in bestFifteen:
        result.append(wordsList[best.position])
    return result

In [6]:
#Devuelve P(y|x1,...,x15)
def probOfBeingSpam(words):
    #**Si/(**Si + **Hi)
    spam_dict, S, ham_dict, H = training('./spam.mbox', './ham.mbox')
    Si = 1
    Hi = 1
    for word in words:
        if word in spam_dict.keys():
            Si = Si*spam_dict[word]
        if word in ham_dict.keys():
            Hi = Hi*ham_dict[word]
    Pyxn = Si/(Si+Hi)
    return Pyxn
    

In [24]:
# Función que recibe un mbox de correos nuevos
# y que devuelva una lista de 0 o 1 para cada correo nuevo 
def clasification(newsbox):
    wordsPerMail = words_per_mail(newsbox)
    clasification = []
    for mailWords in wordsPerMail:
        bestFifteen = bestQualificationWords(mailWords)
        probs = probOfBeingSpam(bestFifteen)
        if probs > 0.9:
            clasification.append(1)
            print("hola")
        else:
            clasification.append(0)
            print("hola2")
    return clasification
    

# Incorporación

In [8]:
# Escribe el nuevo correo ubicado en la ruta 'new'
# en el fichero de ruta 'box'
def append_new_to_mbox(new, mbox):
    with open(mbox, 'a') as writable:
        with open(new) as readable:
            writable.write(readable.read())


# Actualiza dictionary a partir del set de palabras
# del único mensaje contenido en 'new_box'
def update_dict(new_box, dictionary):
    new_words = next(words_per_mail(new_box))
    merge_to_dict(new_words, dictionary)


In [9]:
# Actualiza dictionary y escribe el contenido de new_box en mbox
def incorporation(dictionary, new_box, mbox):
    update_dict(new_box, dictionary)
    append_new_to_mbox(new_box, mbox)


# Uso

In [25]:
# Archivos
newspam = './new-spam.mbox'
newham = './new-ham.mbox'
spambox = './spam.mbox'
hambox = './ham.mbox'

# Entrenamiento
spam_dict, S, ham_dict, H = training(spambox, hambox)

# Clasificación
  #probPerWord
wordsPerMail = words_per_mail('./news.mbox')
print(probPerWord(next(wordsPerMail)))
  #bestQualificationWords
best = bestQualificationWords(next(wordsPerMail))
print(best)
  #probOfBeingSpam
print(probOfBeingSpam(best))  
  #clasification
cla =  clasification('./news.mbox')

# Incorporación a spam.box
incorporation(spam_dict, newspam, spambox)

# Incorporación a ham.box
incorporation(ham_dict, newham, hambox)

[0.0, 0.027586206896551724, 0.0, 0.0, 0.5, 0.375, 0.23076923076923078, 0.027586206896551724, 0.3333333333333333, 0.0, 0.013793103448275862, 0.6, 0.6666666666666666, 0.6781609195402298, 0.75]
['$0', 'quick', 'performance', 'experts', 'last', 'effects', 'better', 'Need', 'erection', 'Sex', 'because:', 'take', 'than', 'at', 'only']
0.9995260360947695
hola2
hola
hola
hola2
hola
hola
hola2
hola2
hola2
hola2
hola2
hola
hola2
hola2
hola2
hola2
hola2
hola2
hola2
hola2
hola2
hola2
hola2
hola2
hola2
hola2
hola2
hola2
hola2
hola2
hola2
hola2
hola2
hola2


KeyboardInterrupt: 